In [ ]:
from pathlib import Path

import Metashape

from result import Ok, Err, Result

from benthoscan.registration import PointCloud, PointCloudLoader
from benthoscan.backends import metashape as backend
from benthoscan.utils.log import logger

In [ ]:
PATHS: dict[str, Path] = {
    "DOCUMENT_IN": Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz"
    ),
    "DOCUMENT_OUT": Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_projects_test/r23685bc_working_version_saved.psz"
    ),
    "CACHE": Path("/home/martin/dev/benthoscan/.cache/"),
}

result: Result[str, str] = backend.load_project(PATHS.get("DOCUMENT_IN"))
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

backend.log_internal_data()

### Define disparity estimator

In [ ]:
import cv2
import numpy as np

def sgbm_default_parameters() -> dict:
    """
    Returns default parameters for semi-global block matching.
    Adopted from: https://github.com/eborboihuc/stereo-matching/blob/master/sgbm.py
    """
    
    min_disparity: int = 0 * 16
    num_disparities: int = 7 * 16 # NOTE: 112
    block_size: int = 10
    window_size: int = 3
    disp12_max_diff: int = 1
    uniqueness_ratio: int = 10
    speckle_window_size: int = 10
    speckle_range: int = 1
    pre_filter_cap: int = 63
    mode: object = cv2.STEREO_SGBM_MODE_SGBM_3WAY
    
    p1: int = 8 * 1 * block_size**2 # **2 # Smoothness
    p2: int = 32 * 1 * block_size**2 # **2 # Smoothness - 

    return {
        "minDisparity": min_disparity,
        "numDisparities": num_disparities,
        "blockSize": block_size,
        "P1": p1,
        "P2": p2,
        "disp12MaxDiff": disp12_max_diff,
        "uniquenessRatio": uniqueness_ratio,
        "speckleWindowSize": speckle_window_size,
        "speckleRange": speckle_range,
        "preFilterCap": pre_filter_cap,
        "mode": mode
    }

def filter_disparity(
    estimator: object, 
    image_left: np.ndarray, 
    disparity_left: np.ndarray, 
    disparity_right: np.ndarray
) -> np.ndarray:
    """Filter disparity map based on WLS. Returns the disparity map as 16-bit pixels."""
    
    filtered_disparity_left = estimator.filter(
        disparity_left, 
        image_left, 
        None, 
        disparity_right,
    )
    
    return filtered_disparity_left
    

def compute_disparity_sgbm(image_left: np.ndarray, image_right: np.ndarray, smooth: bool=False):
    """Computes the disparity for an image pair using OpenCVs SGBM algorithm."""
    
    parameters: dict = sgbm_default_parameters()
    
    left_matcher = cv2.StereoSGBM_create(**parameters)
    right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)

    # Disparity is represented with 16-bit integers. To convert to pixel value cast to float 
    # and divide by 16, i.e.:  disparity.astype(np.float32) / 16.0
    disparity_left: np.ndarray = left_matcher.compute(image_left, image_right)
    disparity_right: np.ndarray = right_matcher.compute(image_right, image_left)

    disparity_left: np.ndarray = disparity_left.astype(np.float32) / 16.0
    disparity_right: np.ndarray = disparity_right.astype(np.float32) / 16.0

    if smooth:
        # FILTER Parameters
        lmbda: int = 100 # regularization parameter 500
        sigma: float = 0.2
    
        disparity_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
        disparity_filter.setLambda(lmbda)
        disparity_filter.setSigmaColor(sigma)
    
        disparity_left: np.ndarray = filter_disparity(disparity_filter, image_left, disparity_left, disparity_right)
        disparity_right: np.ndarray = filter_disparity(disparity_filter, image_right, disparity_right, disparity_left)

    assert disparity_left.dtype == np.float32, "invalid disparity type: expected int16"
    assert disparity_right.dtype == np.float32, "invalid disparity type: expected int16"

    return disparity_left, disparity_right

### Add debug code from simplestereo

In [ ]:
from benthoscan.geometry.stereo import (
    CameraCalibration,
    StereoCalibration,
    RectifyingHomography,
    RectificationResult,
)

def compute_q_matrix(baseline, K1, K2) -> np.ndarray:
    """ """
    b   = baseline
    fx  = K1[0,0]
    fy  = K2[1,1]
    cx1 = K1[0,2]
    cx2 = K2[0,2]
    a1  = K1[0,1]
    a2  = K2[0,1]
    cy  = K1[1,2]
    
    Q = np.eye(4, dtype=np.float64)
    
    Q[0,1] = -a1/fy
    Q[0,3] = a1*cy/fy - cx1
    
    Q[1,1] = fx/fy
    Q[1,3] = -cy*fx/fy
                             
    Q[2,2] = 0
    Q[2,3] = -fx
    
    Q[3,1] = (a2-a1)/(fy*b)
    Q[3,2] = 1/b                        
    Q[3,3] = ((a1-a2)*cy+(cx2-cx1)*fy)/(fy*b)
    
    return Q


def debug_stereo(calibration: StereoCalibration, homographies: RectifyingHomography, result: RectificationResult) -> None:
    """Debug stereo transformations. Based on """
        # Homogeneous transformations
    T1 = np.hstack( (np.eye(3), np.zeros((3, 1))) ) # Camera one has identity rotation and zero translation
    T2 = np.hstack( (calibration.extrinsics.rotation, calibration.extrinsics.location.reshape(3,1)) )
    
    # Original projection matrices
    Po1 = calibration.master.projection.dot(T1)
    Po2 = calibration.slave.projection.dot(T2)

    # Camera centers and baseline
    C1 = np.zeros(3)
    C2 = -np.linalg.inv(Po2[:, :3]).dot(Po2[:,3])
    B = np.linalg.norm(C2)

    Rcommon = homographies.rotation
    
    # Rectified camera matrices
    K1 = result.master.calibration.projection
    K2 = result.slave.calibration.projection
    
    # Rectified projections
    P1 = K1.dot(Rcommon).dot(np.hstack((np.eye(3), -C1[:, None])))
    P2 = K2.dot(Rcommon).dot(np.hstack((np.eye(3), -C2[:, None])))

    Q: np.ndarray = compute_q_matrix(B, K1, K2)
    
    logger.info("---------- Projections ----------")
    logger.info(f"Po1: {Po1}")
    logger.info(f"Po2: {Po2}")
    logger.info("---------------------------------")

    logger.info("---------- Camera centers ----------")
    logger.info(f"C1: {C1}")
    logger.info(f"C2: {C2}")
    logger.info(f"B: {B}")
    logger.info("------------------------------------")
    
    logger.info("---------- Projections ----------")
    logger.info(f"P1: {P1}")
    logger.info(f"P2: {P2}")
    logger.info("---------------------------------")

    logger.info(f"Q: {Q}")

### Define overall stereo depth estimation process

In [ ]:
import cv2

import plotly.express as px
import plotly.graph_objects as go

from benthoscan.backends.metashape.camera_helpers import (
    SensorPair, 
    CameraPair,
    StereoGroup,
    compute_camera_calibration,
    compute_stereo_calibration,
    image_to_array,
    get_stereo_groups,
)

from benthoscan.geometry.stereo import (
    CameraCalibration,
    StereoCalibration,
    RectifyingHomography,
    RectificationResult,
    compute_rectifying_homographies,
    compute_rectifying_pixel_maps,
    rectify_image_pair,
)


def disparity_to_range(disparity_map: np.ndarray, focal_length: float, baseline: float) -> np.ndarray:
    """Converts disparity to range."""
    INVALID_RANGE: float = 0.0
    
    inverse_disparity: np.ndarray = 1.0 / disparity_map
    
    range_map: np.ndarray = inverse_disparity * focal_length * baseline
    range_map: np.ndarray = np.where(range_map < 0.0, INVALID_RANGE, range_map)

    return range_map


def metashape_stereo_processing(
    chunk: Metashape.Chunk, 
    sensor_pair: SensorPair, 
    camera_pairs: list[CameraPair]
) -> None:
    """Process the stereo images in a Metashape chunk."""
    
    # Convert a pair of Metashape sensors to a stereo calibration
    calibration: StereoCalibration = compute_stereo_calibration(sensor_pair)

    # Estimate homographies that transforms the projects to the same plane
    homographies: RectifyingHomography = compute_rectifying_homographies(calibration)

    # Estimate transformations that rectify the image pixels
    result: RectificationResult = compute_rectifying_pixel_maps(calibration, homographies)
        
    for camera_pair in camera_pairs:

        # Convert Metashape images to Numpy arrays
        master_image: np.ndarray = np.squeeze(image_to_array(camera_pair.master.image()))
        slave_image: np.ndarray = np.squeeze(image_to_array(camera_pair.slave.image()))

        # Convert RGB to grayscale
        master_image: np.ndarray = cv2.cvtColor(master_image, cv2.COLOR_RGB2GRAY)
        
        process_stereo_images(
            calibration=calibration,
            homographies=homographies,
            result=result,
            master_image=master_image,
            slave_image=slave_image,
        )
    

def process_stereo_images(
    calibration: StereoCalibration,
    homographies: RectifyingHomography,
    result: RectificationResult,
    master_image: np.ndarray, 
    slave_image: np.ndarray,
) -> None:
    """Get the stereo images"""

    focal_length: float = result.master.calibration.focal_length
    baseline: float = np.linalg.norm(result.slave.location)

    logger.info(f"Focal length:    {focal_length}")
    logger.info(f"Baseline:        {baseline}")

    # Rectify the images
    rectified_master, rectified_slave = rectify_image_pair(
        master_image,
        slave_image,
        result,
    )

    # Compute disparity maps with semi-global block matching
    disparity_maps: tuple[np.ndarray, np.ndarray] = compute_disparity_sgbm(rectified_master, rectified_slave, smooth=False)
    
    # Convert disparity to range estimates
    range_map: np.ndarray = disparity_to_range(disparity_maps[0], focal_length, baseline)

    figures: dict = {
        "master": px.imshow(rectified_master, origin="upper", color_continuous_scale="gray"),
        "slave": px.imshow(rectified_slave, origin="upper", color_continuous_scale="gray"),
        "range": px.imshow(range_map, origin="upper", zmin=1.0, zmax=5.0),
    }

    for key, figure in figures.items():
        figure.show()

    input("Press a key...")
    # raise NotImplementedError("process_stereo_images is not implemented")


def export_stereo_range_maps(chunk: Metashape.Chunk, directory: Path) -> None:
    """Export range maps based on a stereo camera setup."""

    # Get pairs of sensors and cameras (master-slaves)
    stereo_groups: list[StereoGroup] = get_stereo_groups(chunk)

    for group in stereo_groups:
        metashape_stereo_processing(chunk, group.sensor_pair, group.camera_pairs)
    
    raise NotImplementedError

### Test stereo depth estimation on camera pairs from Metashape chunks

In [ ]:
document: Metashape.Document = backend.context._backend_data.get("document")

target_labels: list[str] = [ "r23685bc_20100605_021022" ]
target_chunks: list[Metashape.Chunk] = [chunk for chunk in document.chunks if chunk.label in target_labels]

output_root: Path = Path("/data/kingston_snv_01/stereo_range_maps")

# Generate range maps based on stereo pairs
for chunk in target_chunks:
    output_directory: Path = output_root / Path(f"{chunk.label}_range_maps")
    export_stereo_range_maps(chunk, output_directory)